In [1]:
import sys
sys.path.append('../..')

import pickle
import ml.structures
from ml.dataset.corpus import *
import numpy as np

import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from keras.models import load_model

from keras.layers import LSTM
from keras.layers import Embedding
from keras.optimizers import RMSprop


from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils
from keras.engine.topology import Input

from sklearn.metrics import accuracy_score
from keras.layers import concatenate
from keras.layers.merge import Concatenate

import keras
from keras.utils import to_categorical

from matplotlib import pyplot as plt
from IPython.display import clear_output
DATASET_HOME='../../datasets/'

/home/sharik/.pyenv/versions/3.6.0/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [ ]:
chord_in_bar = 16

In [2]:
def make_16(chord):
    if (chord.duration > 0):
        c = 16 * (chord.duration/128)
    else:
        return 0, None
    array = np.sort([note.number % 12 for note in chord.notes])
    if len(array) == 0:
        array = [0]
    if c >= 1:
        return 1, np.tile(array, (int(c), 1))
    else:
        return 0, None

In [49]:
def cut_song(song, default_notes, default_chords):
    chords_track = []
    notes_track = []
    flag = 0
    for note in song.tracks[0].chords:
        if (make_16(note)[0] == 0):
            flag = 1
            break
        notes_track.append(make_16(note)[1])
    for chord in song.tracks[1].chords:
        if (make_16(chord)[0] == 0):
            flag = 1
            break
        chords_track.append(make_16(chord)[1])
    if flag == 1:
        return 0, default_notes, default_chords
    return 1, notes_track, chords_track

In [4]:
def norm_array(X, size):
    for j in range(len(X)):
        X[j] = np.unique(np.array(X[j]))[:size]
        array = np.zeros(size-len(X[j])) - 1
        X[j] = np.append(X[j], array)
    X = np.array(X)
    return X

In [5]:
def create_X_Y(chords):
    real_chords = []
    y = []
    for i, subchord in enumerate(chords):
        if i != len(chords) - 1:
            real_chords += subchord
            size = len(subchord)
            for j in range(size):
                y.append(chords[i+1][-1])
    return real_chords, y

In [6]:
def notes_to_np(notes):
    for i in range(len(notes)):
        notes[i] = np.array(notes[i])
    notes = np.array(notes)

In [7]:
def create_unique_chords(X, unique_chords):
    for x in X:
        unique_chords = np.append(unique_chords, np.array(x))
    return unique_chords

In [128]:
def encode(X):
    coded_X = np.zeros(len(X))
    for j, k in enumerate(X):
        for i, unique_chord in enumerate(unique_chords):
            if np.array_equal(k, unique_chord):
                coded_X[j] = i
    return coded_X

In [21]:
model_chords_input = Input((334, ), batch_shape=(1,1))
model_chords_embedding = Embedding(794, 20, input_length=1, batch_input_shape=(1,1))(model_chords_input)
model_notes_input = Input((334, ), batch_shape=(1,1))
model_notes_embedding = Embedding(128, 20, input_length=1, batch_input_shape=(1,1))(model_notes_input)

In [22]:
merged = keras.layers.concatenate([model_chords_embedding, model_notes_embedding])

lstm = LSTM(20, stateful=True)(merged)
model_dense = Dense(794, activation='softmax')(lstm)

merged_model = keras.Model(inputs=[model_chords_input, model_notes_input], outputs=model_dense)
merged_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [25]:
corpus = SongCorpus()
corpus.load_from_file(DATASET_HOME+'simple/simple_dataset_SimplifyChordsMapper.pickle')

In [54]:
allX = []
allY = []
unique_chords = np.zeros(4) - 1

for song in tqdm.tqdm_notebook(corpus.songs):
    chords = []
    notes = []
    flag_notes_chords = cut_song(song, notes, chords)
    if (flag_notes_chords[0] == 0) :
        continue
    notes = flag_notes_chords[1]
    chords = flag_notes_chords[2]
    X = []
    Y = []
    X, Y = create_X_Y(chords)
    X = norm_array(X, 4)
    Y = norm_array(Y, 4)
    notes_to_np(notes)
    unique_chords = create_unique_chords(X, unique_chords)
    unique_chords = create_unique_chords(Y, unique_chords)
    unique_chords = unique_chords.reshape((-1, 4))
    unique_chords = np.unique(unique_chords, axis=0)
    unique_chords = list(unique_chords)
    coded_X = []
    coded_Y = []
    coded_X = create_coded(X)
    coded_Y = create_coded(Y)
    encode(X, coded_X)
    encode(Y, coded_Y)
    X = np.array(coded_X)
    y = np.array(coded_Y)
    X_notes = notes
    X_notes = X_notes[:len(X)]
    X = X[:len(X_notes)]
    y = y[:len(X)]
    allX.append([X, X_notes])
    allY.append(y)

In [58]:
# np.save('allX.npy', allX)
# np.save('allY.npy', allY)
with open('allX.pickle', 'wb') as outf:
    pickle.dump(allX, outf)
with open('allY.pickle', 'wb') as outf:
    pickle.dump(allY, outf)
with open('unique_chords.pickle', 'wb') as outf:
    pickle.dump(unique_chords, outf)

In [59]:
with open('allX.pickle', 'rb') as inf:
    allX = pickle.load(inf)
with open('allY.pickle', 'rb') as inf:
    allY = pickle.load(inf)
with open('unique_chords.pickle', 'rb') as inf:
    unique_chords = pickle.load(inf)

In [62]:
unique_chords

[array([-1., -1., -1., -1.]),
 array([ 0., -1., -1., -1.]),
 array([ 0.,  3.,  8., -1.]),
 array([ 0.,  4.,  7., -1.]),
 array([ 0.,  5.,  9., -1.]),
 array([ 1.,  4.,  9., -1.]),
 array([ 1.,  5.,  8., -1.]),
 array([ 1.,  6., 10., -1.]),
 array([ 2.,  5., 10., -1.]),
 array([ 2.,  6.,  9., -1.]),
 array([ 2.,  7., 11., -1.]),
 array([ 3.,  6., 11., -1.]),
 array([ 3.,  7., 10., -1.]),
 array([ 4.,  8., 11., -1.])]

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(allX, allY)

In [46]:
all_len = 0
acc = 0
for i, (x, y) in enumerate(zip(X_train, y_train)):
    print(i)
    merged_model.fit(x, y, epochs=1, batch_size=1, verbose=1)
    acc += merged_model.evaluate(x, y, batch_size=1)[1] * len(y)
    all_len += len(y)
    print("current_accuracy: ", acc / all_len)
    merged_model.reset_states()

Epoch 1/1
318/318 [==============================] - 0s 1ms/step
current_accuracy:  0.1761006289308176
Epoch 1/1
142/142 [==============================] - 0s 711us/step
current_accuracy:  0.16521739130434782
Epoch 1/1
190/190 [==============================] - 0s 760us/step
current_accuracy:  0.20615384615384616
Epoch 1/1
46/46 [==============================] - 0s 689us/step
current_accuracy:  0.21551724137931033
Epoch 1/1
142/142 [==============================] - 0s 591us/step
current_accuracy:  0.22195704057279236
Epoch 1/1
62/62 [==============================] - 0s 583us/step
current_accuracy:  0.24444444444444444
Epoch 1/1
46/46 [==============================] - 0s 940us/step
current_accuracy:  0.27061310782241016
Epoch 1/1
46/46 [==============================] - 0s 891us/step
current_accuracy:  0.25806451612903225
Epoch 1/1
46/46 [==============================] - 0s 840us/step
current_accuracy:  0.2581888246628131
Epoch 1/1
158/158 [==============================] - 0s 847u

62/62 [==============================] - 0s 1ms/step
current_accuracy:  0.3994505494505495
Epoch 1/1
318/318 [==============================] - 0s 828us/step
current_accuracy:  0.3975424022152994
Epoch 1/1
222/222 [==============================] - 0s 824us/step
current_accuracy:  0.4155
Epoch 1/1
62/62 [==============================] - 0s 634us/step
current_accuracy:  0.418343780930386
Epoch 1/1
206/206 [==============================] - 0s 813us/step
current_accuracy:  0.42565411614550097
Epoch 1/1
166/166 [==============================] - 0s 785us/step
current_accuracy:  0.43487721479639413
Epoch 1/1
270/270 [==============================] - 0s 855us/step
current_accuracy:  0.43809665871121717
Epoch 1/1
126/126 [==============================] - 0s 1ms/step
current_accuracy:  0.44216691068814057
Epoch 1/1
94/94 [==============================] - 0s 664us/step
current_accuracy:  0.4423743500866551
Epoch 1/1
158/158 [==============================] - 0s 715us/step
current_accuracy:

94/94 [==============================] - 0s 831us/step
current_accuracy:  0.4869360902255639
Epoch 1/1
78/78 [==============================] - 0s 608us/step
current_accuracy:  0.4868445605523419
Epoch 1/1
66/66 [==============================] - 0s 610us/step
current_accuracy:  0.4873887240356083
Epoch 1/1
252/252 [==============================] - 0s 722us/step
current_accuracy:  0.4917542587894165
Epoch 1/1
318/318 [==============================] - 0s 619us/step
current_accuracy:  0.4998238506253303
Epoch 1/1
62/62 [==============================] - 0s 627us/step
current_accuracy:  0.5021899088997898
Epoch 1/1
142/142 [==============================] - 0s 808us/step
current_accuracy:  0.5053642498702198
Epoch 1/1
94/94 [==============================] - 0s 636us/step
current_accuracy:  0.5053209749399244
Epoch 1/1
110/110 [==============================] - 0s 598us/step
current_accuracy:  0.5065465056963101
Epoch 1/1
254/254 [==============================] - 0s 721us/step
current_

110/110 [==============================] - 0s 1ms/step
current_accuracy:  0.5846774193548387
Epoch 1/1
254/254 [==============================] - 0s 580us/step
current_accuracy:  0.5837884698742957
Epoch 1/1
126/126 [==============================] - 0s 758us/step
current_accuracy:  0.5847559924124849
Epoch 1/1
62/62 [==============================] - 0s 906us/step
current_accuracy:  0.5844010662997678
Epoch 1/1
62/62 [==============================] - 0s 1ms/step
current_accuracy:  0.5840480274442539
Epoch 1/1
126/126 [==============================] - 0s 1ms/step
current_accuracy:  0.5844920242258808
Epoch 1/1
142/142 [==============================] - 0s 807us/step
current_accuracy:  0.5828387315584196
Epoch 1/1
254/254 [==============================] - 0s 719us/step
current_accuracy:  0.5843888935491989
Epoch 1/1
286/286 [==============================] - 0s 859us/step
current_accuracy:  0.5863312334217506
Epoch 1/1
110/110 [==============================] - 0s 815us/step
current_

126/126 [==============================] - 0s 646us/step
current_accuracy:  0.5994342595244916
Epoch 1/1
46/46 [==============================] - 0s 766us/step
current_accuracy:  0.5995209837706442
Epoch 1/1
62/62 [==============================] - 0s 603us/step
current_accuracy:  0.5990155514338708
Epoch 1/1
78/78 [==============================] - 0s 740us/step
current_accuracy:  0.5984918545920183
Epoch 1/1
78/78 [==============================] - 0s 1ms/step
current_accuracy:  0.5987514188422247
Epoch 1/1
126/126 [==============================] - 0s 700us/step
current_accuracy:  0.5982767144572356
Epoch 1/1
126/126 [==============================] - 0s 774us/step
current_accuracy:  0.5978765293207706
Epoch 1/1
142/142 [==============================] - 0s 800us/step
current_accuracy:  0.5986846708178829
Epoch 1/1
246/246 [==============================] - 0s 1ms/step
current_accuracy:  0.5991259711431742
Epoch 1/1
62/62 [==============================] - 0s 1ms/step
current_accura

...

142/142 [==============================] - 0s 661us/step
current_accuracy:  0.6424429540053765
Epoch 1/1
2038/2038 [==============================] - 1s 640us/step
current_accuracy:  0.6428830581233689
Epoch 1/1
206/206 [==============================] - 0s 635us/step
current_accuracy:  0.6429052560996052
Epoch 1/1
126/126 [==============================] - 0s 605us/step
current_accuracy:  0.6429128651442293
Epoch 1/1
270/270 [==============================] - 0s 622us/step
current_accuracy:  0.6428617119463406
Epoch 1/1
46/46 [==============================] - 0s 695us/step
current_accuracy:  0.642863887230595
Epoch 1/1
62/62 [==============================] - 0s 596us/step
current_accuracy:  0.6428762863677756
Epoch 1/1
126/126 [==============================] - 0s 630us/step
current_accuracy:  0.642889985079626
Epoch 1/1
782/782 [==============================] - 0s 623us/step
current_accuracy:  0.643016596664852
Epoch 1/1
46/46 [==============================] - 0s 912us/step
curre

46/46 [==============================] - 0s 768us/step
current_accuracy:  0.643032498178421
Epoch 1/1
606/606 [==============================] - 0s 631us/step
current_accuracy:  0.6429121609375944
Epoch 1/1
126/126 [==============================] - 0s 644us/step
current_accuracy:  0.6428668500828997
Epoch 1/1
62/62 [==============================] - 0s 819us/step
current_accuracy:  0.6428761240993617
Epoch 1/1
158/158 [==============================] - 0s 610us/step
current_accuracy:  0.6428586523685744
Epoch 1/1
62/62 [==============================] - 0s 644us/step
current_accuracy:  0.6428483022604328
Epoch 1/1
126/126 [==============================] - 0s 645us/step
current_accuracy:  0.6428588675236246
Epoch 1/1
126/126 [==============================] - 0s 696us/step
current_accuracy:  0.6428784815474214
Epoch 1/1
94/94 [==============================] - 0s 669us/step
current_accuracy:  0.6428506775043221
Epoch 1/1
158/158 [==============================] - 0s 690us/step
current

142/142 [==============================] - 0s 644us/step
current_accuracy:  0.6428464232866803
Epoch 1/1
254/254 [==============================] - 0s 608us/step
current_accuracy:  0.6427634895108252
Epoch 1/1
62/62 [==============================] - 0s 681us/step
current_accuracy:  0.6427097114845182
Epoch 1/1
46/46 [==============================] - 0s 680us/step
current_accuracy:  0.6427118644067796
Epoch 1/1
126/126 [==============================] - 0s 700us/step
current_accuracy:  0.6426983950835751
Epoch 1/1
46/46 [==============================] - 0s 718us/step
current_accuracy:  0.642678055159011
Epoch 1/1
46/46 [==============================] - 0s 645us/step
current_accuracy:  0.64267271238743
Epoch 1/1
46/46 [==============================] - 0s 675us/step
current_accuracy:  0.6426688696841208
Epoch 1/1
46/46 [==============================] - 0s 759us/step
current_accuracy:  0.6426710244224224
Epoch 1/1
46/46 [==============================] - 0s 700us/step
current_accurac

46/46 [==============================] - 0s 686us/step
current_accuracy:  0.6426880303620665
Epoch 1/1
62/62 [==============================] - 0s 636us/step
current_accuracy:  0.6427001546514395
Epoch 1/1
190/190 [==============================] - 0s 689us/step
current_accuracy:  0.6427312054945218
Epoch 1/1
270/270 [==============================] - 0s 617us/step
current_accuracy:  0.6426516479923916
Epoch 1/1
46/46 [==============================] - 0s 709us/step
current_accuracy:  0.642653784774126
Epoch 1/1
110/110 [==============================] - 0s 673us/step
current_accuracy:  0.6426765275681329
Epoch 1/1
158/158 [==============================] - 0s 635us/step
current_accuracy:  0.6427009716986457
Epoch 1/1
318/318 [==============================] - 0s 640us/step
current_accuracy:  0.6426647782271234
Epoch 1/1
158/158 [==============================] - 0s 610us/step
current_accuracy:  0.6427144405028721
Epoch 1/1
78/78 [==============================] - 0s 651us/step
current

78/78 [==============================] - 0s 605us/step
current_accuracy:  0.6429299160500225
Epoch 1/1
62/62 [==============================] - 0s 615us/step
current_accuracy:  0.642946313427826
Epoch 1/1
46/46 [==============================] - 0s 737us/step
current_accuracy:  0.6429616588304221
Epoch 1/1
254/254 [==============================] - 0s 626us/step
current_accuracy:  0.6429214686804714
Epoch 1/1
510/510 [==============================] - 0s 630us/step
current_accuracy:  0.6429569201542158
Epoch 1/1
638/638 [==============================] - 0s 642us/step
current_accuracy:  0.6430594484695706
Epoch 1/1
1022/1022 [==============================] - 1s 629us/step
current_accuracy:  0.6432219624317561
Epoch 1/1
46/46 [==============================] - 0s 730us/step
current_accuracy:  0.6432152327982728
Epoch 1/1
46/46 [==============================] - 0s 877us/step
current_accuracy:  0.6432246369372197
Epoch 1/1
46/46 [==============================] - 0s 683us/step
current_a

In [ ]:
y_pred = []
for x, y in zip(X_test, y_test):
    y_pred.append(merged_model.predict(x, batch_size=1, verbose=1).argmax(axis=1))

In [73]:
y_real = y_test

In [74]:
from sklearn.metrics import accuracy_score

test_acc = 0
test_len = 0

for y_r, y_p in zip(y_real, y_pred):
    test_acc += accuracy_score(y_r, y_p) * len(y_r)
    test_len += len(y_r)
    
print("test_accuracy:", test_acc / test_len)

test_accuracy: 0.4337806301050175


In [76]:
df = pd.DataFrame(unique_chords)
df.to_csv(DATASET_HOME+"simple/vocab.csv")

In [77]:
x = pd.read_csv(DATASET_HOME+"simple/vocab.csv")
x = x.values.T[1:].T

In [78]:
x

array([[-1., -1., -1., -1.],
       [ 0., -1., -1., -1.],
       [ 0.,  3.,  8., -1.],
       [ 0.,  4.,  7., -1.],
       [ 0.,  5.,  9., -1.],
       [ 1.,  4.,  9., -1.],
       [ 1.,  5.,  8., -1.],
       [ 1.,  6., 10., -1.],
       [ 2.,  5., 10., -1.],
       [ 2.,  6.,  9., -1.],
       [ 2.,  7., 11., -1.],
       [ 3.,  6., 11., -1.],
       [ 3.,  7., 10., -1.],
       [ 4.,  8., 11., -1.]])

In [47]:
merged_model.save(DATASET_HOME+"simple/NN_model.h5")

In [121]:
merged_model = keras.models.load_model(DATASET_HOME+"simple/NN_model.h5")

In [90]:
unique_chords

[array([-1., -1., -1., -1.]),
 array([ 0., -1., -1., -1.]),
 array([ 0.,  3.,  8., -1.]),
 array([ 0.,  4.,  7., -1.]),
 array([ 0.,  5.,  9., -1.]),
 array([ 1.,  4.,  9., -1.]),
 array([ 1.,  5.,  8., -1.]),
 array([ 1.,  6., 10., -1.]),
 array([ 2.,  5., 10., -1.]),
 array([ 2.,  6.,  9., -1.]),
 array([ 2.,  7., 11., -1.]),
 array([ 3.,  6., 11., -1.]),
 array([ 3.,  7., 10., -1.]),
 array([ 4.,  8., 11., -1.])]

In [98]:
new_unique_chords = []
for chord in unique_chords:
    new_unique_chords.append([int(note)+60 for note in chord if int(note) != -1])
unique_chords = new_unique_chords

In [129]:
def encode_song(song):
    flag, notes, chords = cut_song(song, [], [])
    if flag == 0:
        return

    X, Y = create_X_Y(chords)
    X = norm_array(X, 4)
    Y = norm_array(Y, 4)
    notes_to_np(notes)
    
    coded_X = encode(X)
    coded_Y = encode(Y)

    X_notes = notes[:len(X)]
    X_chords = X[:len(X_notes)]
    y = y[:len(X)]
    
    return X_chords, X_notes, y

def predict(model, song):
    X_chords, X_notes, y = encode_song(song)
    
    chords = []
    for (chord, note, pred) in zip(X_chords, X_notes, y):
        predicted_chord_num = merged_model.predict([np.array([chord]), np.array([note])], batch_size=1).argmax()
        chord_notes = unique_chords[predicted_chord_num]
        chord = Chord([Note(number) for number in chord_notes], 64, 128)
        chords.append(chord)
    predicted_song = deepcopy(song)
    predicted_song.chord_track.chords = chords
    return predicted_song

In [127]:
predicted_song = predict(merged_model, corpus.songs[0])

AttributeError: 'numpy.ndarray' object has no attribute 'tracks'